In [1]:
import pandas as pd
import networkx as nx
import random

from functions.sudoku import Sudoku

sociality is the number of rows (randomly chosen) that will come from the group best
memory is the number of rows (randomly chosen) that will come from the personal best
stability is the number of rows left over

In [67]:
class Swarm:
    def __init__(self, sudoku, pop_size = 40, num_neighbors = 4,
                sociality = 5 ,memory = 3, mutation_prob = 0.2, smart_mutation_prob = 0.2):
        import networkx as nx
        self.initial_sudoku = sudoku
        self.pop_size = pop_size
        self.num_neighbors = num_neighbors
        self.sociality = sociality
        self.memory = memory
        self.stability = 9 - sociality - memory
        self.mutation_prob = mutation_prob
        self.smart_mutation_prob = smart_mutation_prob
        self.graph = nx.random_regular_graph(num_neighbors, pop_size)
        self.population = [Sudoku(self.fill_sudoku_random(sudoku)) for _ in range(self.pop_size)]
        self.personal_bests = [x for x in self.population]
        self.group_best = [self.find_group_best(x) for x in range(pop_size)]
        
    def find_group_best(self, node):
        neighbor_sudokus = [self.population[x] for x in [node] + [_ for _ in self.graph[node]]]
        neighbor_costs = [x.cost for x in neighbor_sudokus]
        idx = neighbor_costs.index(min(neighbor_costs))
        return neighbor_sudokus[idx] 

    def fill_row_random(self, row):
        import random
        new_row = list()
        missing_values = set(range(1,10)).difference(set(row))
        rearranged = random.sample(missing_values, len(missing_values))
        for i in range(9):
            if row[i] == 0:
                new_row.append(rearranged.pop())
            else:
                new_row.append(row[i])
        return new_row
        
    def fill_sudoku_random(self, sudoku):
        import random       
        filled_sudoku = list()
        
        for row in sudoku.values:
            filled_sudoku.append(self.fill_row_random(row))
        return filled_sudoku
    
#     def mutate(self, sudoku):
#         import random
#         mutated_rows = random.sample(range(9), random.randint(1,9))
#         mutated = list()
#         for i in range(9):
#             if i in mutated_rows:
#                 mutated.append(self.fill_row_random(self.initial_sudoku.values[i]))
#             else:
#                 mutated.append(sudoku.values[i])
#         return Sudoku(mutated)
    
    def mutate(self, sudoku):
        import random
        sudoku_cost = sudoku.cost
        if sudoku_cost >= 12: 
            mutation_size = random.randint(1,9)
        elif sudoku_cost >= 8:
            mutation_size = 3
        elif sudoku_cost >= 4:
            mutation_size = 2
        elif sudoku_cost >= 1:
            mutation_size = 1
        else: 
            mutation_size = 0
        mutated_rows = random.sample(range(9), mutation_size)
        mutated = list()
        for i in range(9):
            if i in mutated_rows:
                mutated.append(self.fill_row_random(self.initial_sudoku.values[i]))
            else:
                mutated.append(sudoku.values[i])
        return Sudoku(mutated)
    
    def smart_mutate(self, sudoku):
        import random
        errors = sudoku.mark_errors()
        for i in range(9):
            remove = set()
            for x in errors[i]:
                if self.initial_sudoku.values[i][x] != 0:
                    remove.add(x)
            errors[i] = list(set(errors[i]).difference(remove))
        mutated = list()
        for i in range(9):
            new_row = list()
            counter = 0
            swapped = sorted(errors[i])[::-1]
            for j in range(9):
                if j in swapped:
                    new_row.append(sudoku.values[i][swapped[counter]])
                    counter += 1
                else:
                    new_row.append(sudoku.values[i][j])
            mutated.append(new_row)
        return Sudoku(mutated)

    def move(self, node):
        import random
        old_sudoku = self.population[node].values
        old_pb = self.personal_bests[node].values
        old_gb = self.group_best[node].values
        new_sudoku = [None] * 9
        scrambled = random.sample(range(9), 9)
        for i in scrambled[:self.sociality]:
            new_sudoku[i] = old_gb[i]
        for j in scrambled[self.sociality:self.sociality + self.memory]:
            new_sudoku[j] = old_pb[j]
        for k in scrambled[self.sociality + self.memory:]:
            new_sudoku[k] = old_sudoku[k]
        return Sudoku(new_sudoku)
        
    def make_new_population(self):
        mutation_prob = self.mutation_prob
        smart_mutation_prob = self.smart_mutation_prob
        import random
        import numpy as np
        new_pop = [self.move(_) for _ in range(self.pop_size)]
        mutations = random.sample(range(self.pop_size), int(mutation_prob * self.pop_size))
        for i in mutations:
            new_pop[i] = self.mutate(new_pop[i])
        smart_mutations = random.sample(range(self.pop_size), int(smart_mutation_prob * self.pop_size))
        for i in mutations:
            new_pop[i] = self.smart_mutate(new_pop[i])
        new_pb = [new_pop[i] if new_pop[i].cost < self.personal_bests[i].cost else self.personal_bests[i] for i in range(self.pop_size)]
        self.population = new_pop
        new_gb = [self.find_group_best(i) if self.find_group_best(i).cost < self.group_best[i].cost else self.group_best[i] for i in range(self.pop_size)]
        self.personal_bests = new_pb
        self.group_best = new_gb
        
    def change_neighbors(self):
        self.graph = nx.random_regular_graph(self.num_neighbors, self.pop_size)
        self.group_best = [self.find_group_best(x) for x in range(self.pop_size)]

In [68]:
easy = [[0,0,0,0,0,8,5,4,3],
       [0,2,5,6,1,0,7,0,9],
       [0,0,0,0,0,0,0,2,6],
       [0,3,0,0,0,2,0,6,7],
       [0,0,0,5,7,1,0,0,0],
       [7,8,0,4,0,0,0,1,0],
       [2,7,0,0,0,0,0,0,0],
       [1,0,9,0,4,3,2,5,0],
       [4,5,3,8,0,0,0,0,0]]

In [76]:
swarm = Swarm(Sudoku(easy), pop_size= 500, num_neighbors=20, mutation_prob=0.2, sociality=5, memory=3)

In [77]:
counter = 0
prev_best = min([x.cost for x in swarm.population])
#while (min([x.cost for x in swarm.population]) > 0):
while(prev_best > 0):
    swarm.make_new_population()
    print(min([x.cost for x in swarm.population]))
    new_best = min([x.cost for x in swarm.population])
    if prev_best == new_best:
        counter += 1
    else: counter = 0
    prev_best = new_best
    if counter > 40:
        print("Making new neighbors")
        swarm.change_neighbors()
        counter = 0

23
23
23
21
18
16
15
13
11
11
7
7
7
6
5
5
4
4
0


In [54]:
print(swarm.population[[x.cost for x in swarm.population].index(min([x.cost for x in swarm.population]))].values)

[[9, 1, 7, 2, 6, 8, 5, 4, 3], [3, 2, 6, 5, 1, 4, 7, 8, 9], [8, 4, 5, 3, 9, 7, 1, 2, 6], [5, 3, 1, 9, 8, 2, 4, 6, 7], [6, 9, 4, 5, 7, 1, 8, 3, 2], [7, 8, 2, 4, 3, 6, 9, 1, 5], [2, 7, 8, 1, 5, 9, 6, 3, 4], [1, 6, 9, 7, 4, 3, 2, 5, 8], [4, 5, 3, 8, 2, 6, 9, 7, 1]]


In [41]:
print(swarm.initial_sudoku.values)

[[0, 0, 0, 0, 0, 8, 5, 4, 3], [0, 2, 5, 6, 1, 0, 7, 0, 9], [0, 0, 0, 0, 0, 0, 0, 2, 6], [0, 3, 0, 0, 0, 2, 0, 6, 7], [0, 0, 0, 5, 7, 1, 0, 0, 0], [7, 8, 0, 4, 0, 0, 0, 1, 0], [2, 7, 0, 0, 0, 0, 0, 0, 0], [1, 0, 9, 0, 4, 3, 2, 5, 0], [4, 5, 3, 8, 0, 0, 0, 0, 0]]


In [42]:
[x.cost for x in swarm.population]

[25,
 6,
 17,
 6,
 14,
 6,
 6,
 13,
 10,
 13,
 6,
 9,
 6,
 30,
 6,
 6,
 6,
 12,
 6,
 8,
 18,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 35,
 6,
 29,
 6,
 6,
 6,
 6,
 6,
 6,
 42,
 6,
 12,
 6,
 32,
 31,
 6,
 32,
 37,
 6,
 6,
 36,
 40,
 8,
 9,
 27,
 6,
 42,
 6,
 6,
 34,
 6,
 6,
 6,
 6,
 6,
 32,
 6,
 6,
 10,
 33,
 6,
 55,
 6,
 6,
 6,
 9,
 25,
 6,
 6,
 40,
 39,
 6,
 6,
 14,
 6,
 6,
 38,
 6,
 6,
 6,
 8,
 6,
 6,
 6,
 33,
 6,
 6,
 9,
 9,
 13,
 6,
 26,
 6,
 6,
 6,
 6,
 9,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 39,
 6,
 6,
 10,
 9,
 6,
 6,
 6,
 6,
 6,
 30,
 6,
 7,
 6,
 6,
 36,
 6,
 6,
 46,
 6,
 6,
 6,
 6,
 36,
 6,
 20,
 6,
 6,
 6,
 28,
 6,
 6,
 12,
 37,
 6,
 13,
 30,
 36,
 14,
 6,
 44,
 6,
 6,
 6,
 6,
 6,
 6,
 11,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 8,
 6,
 52,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 10,
 6,
 12,
 6,
 6,
 6,
 42,
 13,
 25,
 6,
 6,
 6,
 6,
 21,
 9,
 6,
 9,
 36,
 6,
 6,
 39,
 6,
 10,
 6,
 52,
 6,
 6,
 6,
 6,
 34,
 6,
 6,
 6,
 30,
 32,
 6,
 6,
 9,
 6,
 6,
 11,
 6,
 6,
 32,
 6,
 9,
 10,
 31,
 6,
 6,
 6,


In [43]:
sudoku = Sudoku(swarm.population[[x.cost for x in swarm.population].index(min([x.cost for x in swarm.population]))].values)

In [44]:
sudoku.cost

6

In [45]:
sudoku.mark_errors()

[[0], [3, 6], [4], [4, 6], [0, 7], [1], [7], [1, 3], []]

In [46]:
sudoku.values

[[6, 1, 8, 2, 9, 7, 5, 4, 3],
 [3, 2, 5, 6, 1, 4, 7, 8, 9],
 [9, 4, 7, 3, 8, 5, 1, 2, 6],
 [5, 3, 1, 9, 8, 2, 7, 6, 4],
 [6, 9, 4, 5, 7, 1, 8, 3, 2],
 [7, 8, 2, 4, 3, 6, 9, 1, 5],
 [2, 7, 6, 1, 5, 9, 4, 3, 8],
 [1, 8, 9, 6, 4, 3, 2, 5, 7],
 [4, 5, 3, 7, 2, 8, 6, 9, 1]]

In [57]:
swarm.smart_mutate(sudoku).values

[[6, 1, 8, 2, 9, 7, 5, 4, 3],
 [3, 2, 5, 7, 1, 4, 6, 8, 9],
 [9, 4, 7, 3, 8, 5, 1, 2, 6],
 [5, 3, 1, 9, 7, 2, 8, 6, 4],
 [3, 9, 4, 5, 7, 1, 8, 6, 2],
 [7, 8, 2, 4, 3, 6, 9, 1, 5],
 [2, 7, 6, 1, 5, 9, 4, 3, 8],
 [1, 6, 9, 8, 4, 3, 2, 5, 7],
 [4, 5, 3, 7, 2, 8, 6, 9, 1]]

In [58]:
swarm.initial_sudoku.values

[[0, 0, 0, 0, 0, 8, 5, 4, 3],
 [0, 2, 5, 6, 1, 0, 7, 0, 9],
 [0, 0, 0, 0, 0, 0, 0, 2, 6],
 [0, 3, 0, 0, 0, 2, 0, 6, 7],
 [0, 0, 0, 5, 7, 1, 0, 0, 0],
 [7, 8, 0, 4, 0, 0, 0, 1, 0],
 [2, 7, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 9, 0, 4, 3, 2, 5, 0],
 [4, 5, 3, 8, 0, 0, 0, 0, 0]]